<a href="https://colab.research.google.com/github/leegw1211/begin-ml/blob/main/my_first_tf_code_lgAimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

텐서플로우 튜토리얼을 따라해보면서 약간 배운걸로 만들어본 전체 코드.</br>
학습이 잘 되지 않는다. 학습되었다고 보기가 어렵다.</br>
은닉층의 갯수나 각 층의 뉴런의 갯수, batch size 등을 어떻게 결정해야 할지 모르겠다.

# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from tensorflow.keras import layers

### 데이터 셋 읽어오기

In [ ]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [ ]:
(df_train['is_converted'])

2490      True
39766    False
33331    False
22446    False
55567    False
         ...  
51264    False
16902    False
50314    False
16899    False
52370    False
Name: is_converted, Length: 47439, dtype: bool

In [ ]:
df_train.head(10) # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,...,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,...,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,...,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


In [ ]:
df_test

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,True
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,True
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,False
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,False
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,True
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,False


## 2. 데이터 전처리

필요없어보이는 column 제거하기

In [ ]:
# 도메인 지식이 있어야 필요없는걸 잘 제거할 수 있음
to_del = [
    'customer_idx',
    'customer_country',
    'customer_job',
    'lead_desc_length',
    'customer_country.1',
    'response_corporate',
    'lead_owner', ## 여기까지는 중요하지 않은 것 같아서 제거
    'customer_country',
    'inquiry_type',
    'product_category',
    'product_subcategory',
    'product_modelname',
    'customer_position',
    'expected_timeline',
    'business_subarea', ## 이것들은 범주형 변수인데 너무 종류가 많아서 일단 제거
    'id_strategic_ver',
    'idit_strategic_ver' ## 얘네둘은 그냥 전부다 1이라 의미가 없음
]
df_train.drop(to_del, axis=1, inplace=True)
df_test.drop(to_del, axis=1, inplace=True)

nan 처리하기

In [ ]:
# 일단 평균값으로 대체했다.

# DataFrame에서 숫자 열과 문자열 열 나누기
numeric_cols = df_train.select_dtypes(include='number')
string_cols = df_train.select_dtypes(include='object')

# 숫자 열의 NaN 값을 최빈값으로 대체하기
numeric_mode_values = numeric_cols.mode().iloc[0]
df_train[numeric_cols.columns] = numeric_cols.fillna(numeric_mode_values)

# 문자열 열의 NaN 값을 최빈값으로 대체하기
string_mode_values = string_cols.mode().iloc[0]
df_train[string_cols.columns] = string_cols.fillna(string_mode_values)

df_train.sort_index(inplace=True)

# 최빈값으로 대체된 DataFrame 출력
df_train

,bant_submit,business_unit,com_reg_ver_win_rate,customer_type,enterprise,historical_existing_cnt,it_strategic_ver,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,is_converted
0,1.00,AS,0.066667,End-Customer,Enterprise,4.0,1.0,1,0,0.003079,0.026846,corporate / office,True
1,1.00,AS,0.066667,End-Customer,Enterprise,12.0,1.0,1,0,0.003079,0.026846,corporate / office,True
2,1.00,AS,0.088889,End-Customer,Enterprise,144.0,1.0,1,0,0.003079,0.026846,corporate / office,True
3,1.00,AS,0.088889,End-Customer,Enterprise,4.0,1.0,1,0,0.003079,0.026846,corporate / office,True
4,1.00,AS,0.088889,Specifier/ Influencer,Enterprise,4.0,1.0,0,0,0.003079,0.026846,corporate / office,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,AS,0.074949,End Customer,SMB,4.0,1.0,0,0,0.000026,0.028777,public facility,False
59295,0.75,AS,0.040000,Specifier/ Influencer,Enterprise,4.0,1.0,0,0,0.000026,0.028777,public facility,False
59296,0.75,AS,0.040000,Specifier/ Influencer,Enterprise,4.0,1.0,0,0,0.000026,0.028777,public facility,False
59297,1.00,AS,0.040000,End-Customer,Enterprise,4.0,1.0,0,0,0.000026,0.028777,public facility,False


In [ ]:
# nan 처리하기. 일단 평균값으로 대체했다.

# DataFrame에서 숫자 열과 문자열 열 나누기
numeric_cols = df_test.select_dtypes(include='number')
string_cols = df_test.select_dtypes(include='object')

# 숫자 열의 NaN 값을 최빈값으로 대체하기
numeric_mode_values = numeric_cols.mode().iloc[0]
df_test[numeric_cols.columns] = numeric_cols.fillna(numeric_mode_values)

# 문자열 열의 NaN 값을 최빈값으로 대체하기
string_mode_values = string_cols.mode().iloc[0]
df_test[string_cols.columns] = string_cols.fillna(string_mode_values)

# 최빈값으로 대체된 DataFrame 출력
df_test

,id,bant_submit,business_unit,com_reg_ver_win_rate,customer_type,enterprise,historical_existing_cnt,it_strategic_ver,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,is_converted
0,19844,0.00,ID,0.073248,End Customer,Enterprise,53.0,1.0,1,0,0.001183,0.049840,retail,True
1,9738,0.25,IT,0.044643,End Customer,SMB,0.0,1.0,0,0,0.000013,0.064566,transportation,True
2,8491,1.00,ID,0.044643,Specifier/ Influencer,SMB,0.0,1.0,0,0,0.000060,0.131148,hospital & health care,True
3,19895,0.50,ID,0.118644,End Customer,Enterprise,0.0,1.0,0,0,0.001183,0.049840,retail,False
4,10465,1.00,ID,0.074949,End Customer,Enterprise,2.0,1.0,1,1,0.003079,0.064566,corporate / office,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,AS,0.044643,End Customer,Enterprise,10.0,1.0,0,0,0.003079,0.064566,corporate / office,False
5267,7979,0.25,IT,0.044643,End Customer,Enterprise,0.0,1.0,0,0,0.003079,0.064566,corporate / office,False
5268,12887,0.75,AS,0.044643,Specifier/ Influencer,Enterprise,0.0,1.0,0,0,0.003079,0.064566,corporate / office,True
5269,17530,0.00,IT,0.044643,End Customer,SMB,0.0,1.0,0,0,0.003079,0.064566,corporate / office,False


In [ ]:
df_train, df_val = train_test_split(df_train, test_size=0.2)
df_test = df_test.drop('id', axis=1)

dataframe -> dataset

In [ ]:
# pandas dataframe을 dataset으로 변환해주는 함수
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('is_converted')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
batch_size = 64
train_ds = df_to_dataset(df_train, batch_size=batch_size)
val_ds = df_to_dataset(df_val, batch_size=batch_size)
test_ds = df_to_dataset(df_test, batch_size=batch_size)

모델에 넣을 수 있게 정규화 / 인코딩

In [ ]:
def get_normalization_layer(name, dataset):
  normalizer = layers.Normalization(axis=None)
  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)

  return normalizer

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  if dtype == 'string':
    index = layers.StringLookup(max_tokens=max_tokens)
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  feature_ds = dataset.map(lambda x, y: x[name])
  index.adapt(feature_ds)

  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size(), output_mode='one_hot')

  return lambda feature: encoder(index(feature))

In [ ]:
[(train_features, label_batch)] = train_ds.take(1)
train_features['bant_submit']

2024-02-24 11:32:15.681231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int64 and shape [47439]
	 [[{{node Placeholder/_8}}]]
2024-02-24 11:32:15.683160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype double and shape [47439]
	 [[{{node Placeholder/_10}}]]


<tf.Tensor: shape=(64,), dtype=float64, numpy=
array([0.5 , 0.75, 0.5 , 1.  , 1.  , 0.5 , 0.  , 0.75, 0.75, 1.  , 1.  ,
       0.75, 0.5 , 1.  , 0.75, 0.25, 1.  , 1.  , 1.  , 0.5 , 0.5 , 1.  ,
       0.5 , 0.75, 0.5 , 1.  , 0.25, 0.5 , 0.75, 1.  , 0.5 , 0.75, 1.  ,
       0.75, 0.75, 0.75, 1.  , 0.5 , 0.5 , 0.25, 0.5 , 1.  , 1.  , 0.5 ,
       0.5 , 1.  , 1.  , 0.75, 1.  , 0.5 , 0.5 , 1.  , 0.75, 0.75, 0.25,
       1.  , 0.75, 0.5 , 0.25, 0.5 , 1.  , 0.75, 1.  , 0.5 ])>

In [ ]:
df_train.columns

Index(['bant_submit', 'business_unit', 'com_reg_ver_win_rate', 'customer_type',
       'enterprise', 'historical_existing_cnt', 'it_strategic_ver', 'ver_cus',
       'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'is_converted'],
      dtype='object')

In [ ]:
numeric_columns = ["bant_submit", "com_reg_ver_win_rate", "historical_existing_cnt", "ver_win_rate_x", "ver_win_ratio_per_bu"]
int_categorical_columns= ["ver_cus", "ver_pro"]
str_categorical_columns= ["business_unit", "customer_type", "enterprise", "business_area"]

In [ ]:
all_inputs = []  # 모든 입력 레이어를 저장할 리스트
encoded_features = []  # 처리된 특성을 저장할 리스트

# Numerical features.
for header in numeric_columns:  # 숫자형 특성에 대해서 반복
  # tf.keras.Input을 사용하여 특성에 대한 입력 레이어를 생성
  numeric_col = tf.keras.Input(shape=(1,), name=header)

  # 특성에 대한 정규화 레이어를 가져와서 적용
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)

  # 입력 레이어와 처리된 특성을 리스트에 추가
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

2024-02-24 11:32:22.181005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype double and shape [47439]
	 [[{{node Placeholder/_10}}]]
2024-02-24 11:32:22.182045: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype int64 and shape [47439]
	 [[{{node Placeholder/_9}}]]
2024-02-24 11:32:23.946852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [4

In [ ]:
for header in int_categorical_columns:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
  encoding_layer = get_category_encoding_layer(name=header,
                                               dataset=train_ds,
                                               dtype='int64'
                                               )
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

2024-02-24 11:32:30.002699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype bool and shape [47439]
	 [[{{node Placeholder/_12}}]]
2024-02-24 11:32:30.003462: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [47439]
	 [[{{node Placeholder/_5}}]]
2024-02-24 11:32:31.556648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [47

In [ ]:
for header in str_categorical_columns:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(name=header,
                                               dataset=train_ds,
                                               dtype='string'
                                               )
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

2024-02-24 11:32:32.949788: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int64 and shape [47439]
	 [[{{node Placeholder/_8}}]]
2024-02-24 11:32:32.950524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype int64 and shape [47439]
	 [[{{node Placeholder/_9}}]]
2024-02-24 11:32:34.421501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [4743

In [ ]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(128, activation="relu")(all_features)
x = tf.keras.layers.Dense(64, activation="relu")(all_features)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
total_samples = 47000
positive_samples = 3870  # true인 샘플의 수
negative_samples = total_samples - positive_samples  # false인 샘플의 수
weight_for_0 = (1 / negative_samples) * (total_samples / 2.0)  # false에 대한 가중치
weight_for_1 = (1 / positive_samples) * (total_samples / 2.0)  # true에 대한 가중치

class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(train_ds, epochs=10, validation_data=val_ds, class_weight=class_weight)

Epoch 1/10


2024-02-24 11:49:50.626150: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [47439]
	 [[{{node Placeholder/_0}}]]
2024-02-24 11:49:50.627391: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [47439]
	 [[{{node Placeholder/_5}}]]


742/742 [==============================] - 3s 3ms/step - loss: 0.5744 - accuracy: 0.8374 - val_loss: 0.6174 - val_accuracy: 0.8237
Epoch 2/10
742/742 [==============================] - 3s 4ms/step - loss: 0.5636 - accuracy: 0.8362 - val_loss: 0.5590 - val_accuracy: 0.8356
Epoch 3/10
742/742 [==============================] - 3s 4ms/step - loss: 0.5614 - accuracy: 0.8340 - val_loss: 0.5490 - val_accuracy: 0.8376
Epoch 4/10
742/742 [==============================] - 4s 4ms/step - loss: 0.5598 - accuracy: 0.8323 - val_loss: 0.5651 - val_accuracy: 0.8344
Epoch 5/10
742/742 [==============================] - 3s 4ms/step - loss: 0.5588 - accuracy: 0.8356 - val_loss: 0.6215 - val_accuracy: 0.8143
Epoch 6/10
742/742 [==============================] - 4s 4ms/step - loss: 0.5580 - accuracy: 0.8317 - val_loss: 0.5728 - val_accuracy: 0.8313
Epoch 7/10
742/742 [==============================] - 3s 4ms/step - loss: 0.5574 - accuracy: 0.8318 - val_loss: 0.5363 - val_accuracy: 0.8272
Epoch 8/10
742/74

In [ ]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(test_ds)
print('Test accuracy:', test_accuracy)

34/83 [===========>..................] - ETA: 0s - loss: 0.6068 - accuracy: 0.7688

2024-02-24 11:33:22.929559: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype string and shape [5271]
	 [[{{node Placeholder/_4}}]]
2024-02-24 11:33:22.930838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int64 and shape [5271]
	 [[{{node Placeholder/_8}}]]


83/83 [==============================] - 0s 3ms/step - loss: 0.5763 - accuracy: 0.7813
Test accuracy: 0.7812559008598328


In [ ]:
#모델 성능 보기

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
pred = model.predict(train_ds)

# 확률 값들을 임계값을 기준으로 True 또는 False로 변환
pred = np.where(pred >= 0, True, False)

len(pred)

742/742 [==============================] - 2s 3ms/step


47439

In [ ]:
val_label = []
for x_batch, y_batch in train_ds:
    for y in y_batch.numpy():
        val_label.append(y)
val_label = np.array(val_label)
len(val_label)

47439

In [ ]:
get_clf_eval(val_label, pred)

오차행렬:
 [[   14  3859]
 [  101 43465]]

정확도: 0.9165
정밀도: 0.1217
재현율: 0.0036
F1: 0.0070


## 4. 제출하기

### 테스트 데이터 예측

In [ ]:
# 예측에 필요한 데이터 분리
df_test = df_test.drop(["is_converted"], axis=1)

In [ ]:
test_pred = model.predict(df_test)
sum(test_pred) # True로 예측된 개수

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**